# **혼자 공부하는 머신러닝 + 딥러닝 CH5**

## **교차 검증과 그리드 서치**

### **Import**

In [39]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import uniform, randint

### **데이터 전처리**

In [2]:
wine = pd.read_csv('https://bit.ly/wine-date')

In [3]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [6]:
train_input, test_input, train_target, test_target = train_test_split(data, target, random_state = 42, test_size = 0.2)

In [7]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, random_state = 42, test_size=0.2)

In [8]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


### **트리 분석**

In [10]:
dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target)) # 과대적합 된 상황

0.9971133028626413
0.8634615384615385


In [12]:
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01076508, 0.00745463, 0.00807357, 0.00815558, 0.00714779]), 'score_time': array([0.00100923, 0.0008781 , 0.00085473, 0.00104117, 0.00083137]), 'test_score': array([0.86826923, 0.85384615, 0.87584216, 0.85274302, 0.84023099])}


In [14]:
print(np.mean(scores['test_score']))

0.8581863108018066


In [18]:
splitter = StratifiedKFold(n_splits = 10, shuffle=True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8581895657329184


### **그리드 서치**

In [20]:
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [22]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [23]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [24]:
print(dt.score(test_input, test_target))

0.8653846153846154


In [27]:
print(gs.cv_results_)

{'mean_fit_time': array([0.0093689 , 0.00896478, 0.00851688, 0.00813751, 0.00655313]), 'std_fit_time': array([0.00041427, 0.00107817, 0.00104943, 0.0015045 , 0.00044127]), 'mean_score_time': array([0.00127525, 0.0010097 , 0.00094934, 0.00105557, 0.0008924 ]), 'std_score_time': array([3.09729663e-04, 4.89535528e-05, 2.04125294e-05, 8.14715881e-05,
       1.01572135e-04]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216,

In [25]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [26]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [28]:
params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)}

In [29]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [30]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [31]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### **랜덤 서치**

In [33]:
rgen = randint(0,10)
rgen.rvs(10)

array([4, 2, 4, 0, 2, 9, 9, 6, 6, 5])

In [37]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97,  96,  92,  88, 112, 103, 108,  99, 109,  96]))

In [38]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),
          'max_depth':randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25)}

In [40]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params, n_iter=100,n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [41]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [42]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [45]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
